In [1]:
import numpy as np
import pickle

with open("/kaggle/input/mai-kinda-processed/X.pickle", "rb") as p:
    X = pickle.load(p)
with open("/kaggle/input/mai-kinda-processed/y.pickle", "rb") as p:
    y = pickle.load(p)

In [2]:
embeddings_dict = {}

with open("/kaggle/input/mai-kinda-processed/mai_w2v.txt", "r") as f:
    first = True
    for line in f:
        if first:
            first = False
        else:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:])
            embeddings_dict[word] = coefs

In [3]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tok_obj = Tokenizer()
tok_obj.fit_on_texts(X)
sequences = tok_obj.texts_to_sequences(X)

X = pad_sequences(sequences)

print("Found ", len(tok_obj.word_index), " unique tokens.")
print("Shape of data tensor: ", X.shape)

Found  3954  unique tokens.
Shape of data tensor:  (824, 543)


In [4]:
import h5py

h = h5py.File("data.hdf5", "w")
h.create_dataset("X", data=X, shape=X.shape, compression="gzip")

<HDF5 dataset "X": shape (824, 543), type "<i4">

In [5]:
num_words = len(tok_obj.word_index) + 1
embedding_matrix = np.zeros((num_words, 100), dtype=np.float16)
for word, i in tok_obj.word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(embedding_matrix.shape)

(3955, 100)


In [6]:
h.create_dataset("embedding_matrix", data=embedding_matrix, shape=embedding_matrix.shape, compression="gzip")
h.close()

In [7]:
import gc

del X
del embedding_matrix
gc.collect()

84

In [8]:
tok_obj.fit_on_texts(y)
y = tok_obj.texts_to_sequences(y)

print("Found ", len(tok_obj.word_index), " unique tokens.")

Found  13857  unique tokens.


In [9]:
import pickle

with open("y.pickle", "wb") as p:
    pickle.dump(y, p)

In [10]:
del y
gc.collect()

42